<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/182-Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain이란?
* Language : Large Language Model의 Language
* Chain: 모듈들을 체인으로 연결하여 다른 툴의 출력을 다음 툴의 입력으로 연결하는 방식
  * 예: 검색 -> OpenAI(LLM) -> 계산
* LLM(Large Language Model)을 이용해서 뭔가 만들때 쓸수 있는 툴/라이브러리 모음.
* 파이썬과 자바스크립트 라이브러리를 제공.
* 검색, 메모리, 계산, 요약, 색인, Agent(에이전트) 등의 기능을 제공
* 모듈화로 쉽게 부품을 갈아끼울수 있음
* LLM은 기본 OpenAI (text-davinci-003) 이용

---
참고 자료:
* 파이썬 문서: https://python.langchain.com/en/latest/index.html
* 개념 문서: https://docs.langchain.com/docs/
* 소스: https://github.com/hwchase17/langchain

---

In [ ]:
#@title 기본 패키지(openai, langchain) 설치
!pip install openai
!pip install langchain

In [ ]:
#@title 기타 패키지 설치 (구글검색, 위키피디아, VectorStore, HuggingFace Embedding)
!pip install google-search-results
!pip install wikipedia
!pip install faiss-cpu # 오픈소스 벡터DB (Facebook, MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # Summarization 할때 필요

In [ ]:
#@title 0. API 키 설정
import os
#@markdown https://platform.openai.com/account/api-keys
OPENAI_API_KEY = "sk-" #@param {type:"string"}

#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
HUGGINGFACEHUB_API_TOKEN = "hf_" #@param {type:"string"}

#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
SERPAPI_API_KEY = "" #@param {type:"string"}

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY


In [ ]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

In [ ]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)



In [ ]:
type(llm)

In [ ]:
#@title 1. ChatOpenAI LLM (gpt-3.5-turbo)

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
#@title 2. ChatPromptTemplate & chain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)

In [ ]:
prompt.format(상품="신발")

In [ ]:
chat = ChatOpenAI(temperature=0)

template="You are a helpful assisstant that tranlates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

In [ ]:
prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)

In [ ]:
prompt.format(상품="신발")

In [ ]:
chat = ChatOpenAI(temperature=0)

template="You are a helpful assisstant that tranlates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chatchain = LLMChain(llm=chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Korean", text="Thanks god TGIF!")


In [ ]:
#@title 3. Agents and Tools

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

* Tool: 특정 일을 하는 역할. 예) 구글 검색, 디비 조회, Python 실행/계산.
* Agents: 어떤 툴을 어떤 순서로 실행할지 결정하는 역할 (LLM 이용)

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=chat)
# tools = load_tools(["wikipedia", "llm-math"], llm=chat)

agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("한국의 'New Jeans' 그룹 멤버의 인원 수는? 오늘 2023년 11월 17일 현재 전체 멤버들의 나이의 합은?") # 한국의 'New Jeans' 그룹 멤버의 인원 수는? 오늘 2023년 5월 30일 현재 전체 멤버들의 나이의 합은 얼마인가요

In [ ]:
len(agent.tools)

In [ ]:
print(agent.tools[0].description)
print(agent.tools[1].description)

In [ ]:
#@title 4. Memory
from langchain import ConversationChain

conversation = ConversationChain(llm=chat, verbose=True)
conversation.predict(input="인공지능에서 Transformer가 뭐야?")

In [ ]:
conversation.predict(input="RNN하고 차이 설명해줘.")

In [ ]:
conversation.predict(input="Seq2Seq 와도 차이점을 비교해줘.")

In [ ]:
conversation.memory

* ConversationBufferMemory : 대화 기록(기본)
* ConversationBufferWindowMemory : 마지막 n개의 대화만 기억
* Entity Memory : 개체에 대한 정보를 저장
* Conversation Knowledge Graph Memory: 개체의 triple 저장: (sam, 좋아하는 색, 파랑)
* ConversationSummaryMemory : 대화의 요약본을 저장
* ConversationSummaryBufferMemory : 대화 요약본 + 마지막 n토큰 기억
* ConversationTokenBufferMemory : 마지막 n토큰 기억
* VectorStore-Backed Memory : 벡터DB에 정보 저장

In [ ]:
#@title 5. Document Loaders
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/NewJeans")

documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

# 4096 token = 3000 English word

In [ ]:
print(docs[-1].page_content)

---
* Web Page
* PDF
* Email
* Twitter
* DataFrame (pandas)
* YouTube
* Notion
* Google Drive
* Powerpoint, Word,
* ...
---

In [ ]:
#@title 6. Summarization
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:-1])


* "stuff": LLM 한번에 다 보냄. 길면 오류남
* "map_reduce": 나눠서 요약, 전체 요약본 다시 요약
* "refine": (요약 + 다음 문서) => 요약
* "map_rerank": 점수매겨서 중요한거로 요약

In [ ]:
#@title 7. Embeddings and VectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    # text_splitter=text_splitter,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("faiss-nj")

* Embedding : LLM에 input(text)를 넣었을때 나오는 특징 벡터 - Neural Network의 특정 메모리를 읽어서 만들고, 보통 512개 이상의 float로 만들어짐.
* 의미상 비슷한 input이 들어갔을때, 나오는 특징 벡터가 유사한 특징을 갖고, 거리상 가깝다.
* OpenAI에서 text를 넣으면 Embedding을 돌려주는 API를 제공한다. (다만 비용 발생)
* 로컬에서 유사한 알고리즘을 돌릴수 있다. (다만 똑같은 벡터는 아니고, openAI 임베딩과는 비교할수 없다)
* HuggingFaceEmbeddings 사용하면 로컬에서 돌릴수 있고, GPU 있다면 가속도 지원함.
* Vector Database는 Embedding 벡터와 텍스트를 저장하는 DB
  * Pinecone 서비스 (유료, 무료 제한적)
  * FAISS (페이스북, 오픈소스, 로컬)
  * Embeddings -> 텍스트를 저장
  * Embeddings로 검색할수 있으며, 거리상 가장 가까운 항목을 가져올수 있음. (거리를 구하는 방법은 여러가지)
  * 유사한 의미를 갖는 텍스트에 대한 Embedding 벡터는 거리상 가깝다.

In [ ]:
index.query("뉴진스의 데뷔곡은?", llm=chat, verbose=True)

In [ ]:
index.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)

In [ ]:
index.query("멤버의 나이는?", llm=chat, verbose=True)

In [ ]:
index.query("멤버의 나이는? (오늘은 2023년 11월 18일)", llm=chat, verbose=True)

In [ ]:
#@title FAISS 벡터DB 디스크에서 불러오기
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss-nj", embeddings)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [ ]:
index2.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)